# Time Period: 20150630-20150930

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)

### X1

In [3]:
BEME_ratio_df_2015 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Three_Factor_Model/2015_Q1/BEME_ratio_df_2015.csv", header=0)

In [4]:
norm_BEME_ratio_list = list(BEME_ratio_df_2015['norm_BEME_ratio'])

### X2: 2015Q2 return

In [5]:
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]
# Store files to dictionary using their own name
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

In [6]:
start_close = [None]*len(BEME_ratio_df_2015['WINDCODE'])
end_close = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [7]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    company = BEME_ratio_df_2015['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20150331]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20150630]['S_DQ_ADJCLOSE'])

In [8]:
com_qtr_return = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [9]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

In [10]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [12]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2015-03-30']['last_price'].mean()) # index price on start day
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2015-06-30']['last_price'].mean())  

In [13]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [14]:
print index_qtr_return

0.0940609618148


In [15]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [16]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [17]:
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'diff_csi300_LAST': diff_return_com_csi300})

In [18]:
# Normalize Function
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [19]:
Weighted_Return['norm_diff_csi300_LAST'] = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [20]:
Weighted_Return.to_csv("LAST_Weighted_Return_2015_Q3.csv")

### X3

In [21]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

start_dateind = list(dates_trade).index(20150630) # index of start date
end_dateind = list(dates_trade).index(20150930) # index of end date

In [22]:
target_dates = dates_trade[start_dateind:end_dateind+1]

In [23]:
target_dates

array([20150630, 20150701, 20150702, 20150703, 20150706, 20150707,
       20150708, 20150709, 20150710, 20150713, 20150714, 20150715,
       20150716, 20150717, 20150720, 20150721, 20150722, 20150723,
       20150724, 20150727, 20150728, 20150729, 20150730, 20150731,
       20150803, 20150804, 20150805, 20150806, 20150807, 20150810,
       20150811, 20150812, 20150813, 20150814, 20150817, 20150818,
       20150819, 20150820, 20150821, 20150824, 20150825, 20150826,
       20150827, 20150828, 20150831, 20150901, 20150902, 20150907,
       20150908, 20150909, 20150910, 20150911, 20150914, 20150915,
       20150916, 20150917, 20150918, 20150921, 20150922, 20150923,
       20150924, 20150925, 20150928, 20150929, 20150930])

In [24]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [25]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df_2015['WINDCODE']]

In [26]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'FAST_mean':FAST_mean_list})
FAST_mean_df['FAST_mean_norm'] = normalize_list(FAST_mean_df['FAST_mean'])

In [28]:
FAST_mean_df[:10]

,Company,FAST_mean,FAST_mean_norm
0,000157.SZ,3.804816e+06,-0.241393
1,600887.SH,1.019532e+07,0.147077
2,600276.SH,8.781451e+06,0.061130
3,600009.SH,3.101536e+06,-0.284145
4,601328.SH,2.422352e+07,0.999833
5,600150.SH,5.501954e+06,-0.138226
6,000728.SZ,4.230943e+06,-0.215490
7,600036.SH,3.639358e+07,1.739636
8,600660.SH,2.610169e+06,-0.314014
9,601998.SH,2.382825e+07,0.975805


In [29]:
FAST_mean_df.to_csv("FAST_mean_2015_Q3.csv") # save it as csv

### Y

In [30]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df_2015['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [31]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    company = BEME_ratio_df_2015['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20150630]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20150930]['S_DQ_ADJCLOSE'])

In [32]:
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df_2015['WINDCODE']) 

In [33]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [34]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2015-06-30']['last_price'].mean()) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2015-09-30']['last_price'].mean()) 

In [35]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [36]:
index_qtr_return_CURRENT

-0.283879232004793

In [37]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df_2015['WINDCODE'])

In [38]:
for i in range(len(BEME_ratio_df_2015['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [39]:
diff_return_com_csi300_CURRENT[0:10]

[-0.09599235839311326,
 0.09764091293814664,
 0.3209183932610733,
 0.16871840205639022,
 0.02193655684555723,
 -0.029535041018600494,
 -0.29377335015483147,
 0.26855208105167605,
 0.11371446460828655,
 0.04461542832381146]

In [40]:
Y_2015_Q3 = pd.DataFrame({'Company':BEME_ratio_df_2015['WINDCODE'],'Y':diff_return_com_csi300_CURRENT})

In [41]:
Y_2015_Q3.to_csv("Y_2015_Q3.csv")

In [42]:
TFM_2015_Q3 = pd.DataFrame({'X_1':norm_BEME_ratio_list,'X_2':Weighted_Return['norm_diff_csi300_LAST'], 'X_3': FAST_mean_df['FAST_mean_norm'], 'Y':diff_return_com_csi300_CURRENT})

In [43]:
TFM_2015_Q3.to_csv("TFM_2015_Q3.csv")

In [44]:
def outlier_index(list_name, std_num):
    mean = list_name.mean() # mean
    std = np.std(list_name, axis=0) # standard deviation
    
    upper_b = mean+std_num*std
    lower_b = mean-std_num*std
    
    ind_upper = [i for i in range(len(list_name)) if list_name[i] > upper_b]
    ind_lower = [i for i in range(len(list_name)) if list_name[i] < lower_b]
    outlier_index = ind_upper+ind_lower
    
    return outlier_index


In [45]:
outlier_all =outlier_index(TFM_2015_Q3['X_1'], 5) + outlier_index(TFM_2015_Q3['X_2'], 5)+ outlier_index(TFM_2015_Q3['X_3'], 5)+ outlier_index(TFM_2015_Q3['Y'], 5)

In [46]:
TFM_2015_Q3_NO = TFM_2015_Q3.drop(TFM_2015_Q3.index[outlier_all])

In [47]:
TFM_2015_Q3_NO.to_csv("TFM_2015_Q3_NO.csv")

# ************* Result DataFrame: TFM_2015_Q3_NO.csv *************